In [62]:
import numpy as np

In [99]:
# Load preprocessed dev set examples
X = np.load("../data/interim/X.npy")
X = np.swapaxes(X,1,2)
Y = np.load("../data/interim/Y.npy")
Y = np.swapaxes(Y,1,2)
Y = to_categorical(Y)

In [100]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.utils import to_categorical

In [101]:
def seq_model(input_shape, n_classes):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
        
    # Step 1: CONV layer (≈4 lines)
    X = Conv1D(196, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)

    # Step 2: First GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X) # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization
    
    # Step 3: Second GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X)   # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                  # Batch normalization
    X = Dropout(0.8)(X)                                  # dropout (use 0.8)
    
    # Step 4: Time-distributed dense layer (≈1 line)
    X = TimeDistributed(Dense(n_classes, activation = "sigmoid"))(X) # time distributed  (sigmoid)

    model = Model(inputs = X_input, outputs = X)
    
    return model

In [102]:
Tx = X.shape[1]
n_freq = X.shape[2]
n_classes = Y.shape[2]
model = seq_model((Tx, n_freq), n_classes)

In [103]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 4797, 101)         0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 1196, 196)         297136    
_________________________________________________________________
batch_normalization_31 (Batc (None, 1196, 196)         784       
_________________________________________________________________
activation_11 (Activation)   (None, 1196, 196)         0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 1196, 196)         0         
_________________________________________________________________
gru_21 (GRU)                 (None, 1196, 128)         124800    
_________________________________________________________________
dropout_42 (Dropout)         (None, 1196, 128)         0         
__________

In [104]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [105]:
X.shape, Y.shape

((1000, 4797, 101), (1000, 1196, 21))

In [110]:
model.fit(X, Y, batch_size = 1000, epochs=50)

Epoch 1/50


KeyboardInterrupt: 

In [107]:
X.shape

(1000, 4797, 101)